In [ ]:
# COMMIT MSG
# 
# [Forecasting] - First forecasting notebook working
# TODO! pin versions of new pip dependencies of OJ Sales Forecasting notebook

import azureml.core
from azureml.core import Workspace, Dataset, Datastore, Environment
from azureml.core import Experiment
import azureml.interpret
from azureml.widgets import RunDetails

# Forecasting Specific
from azureml.automl.core.forecasting_parameters import ForecastingParameters

import pandas as pd
import numpy as np

print("SDK version:", azureml.core.VERSION)

In [ ]:
# Get the Workspace object from Azure
from azureml.core.authentication import InteractiveLoginAuthentication

# You can find tenant id under azure active directory->properties
tenant_id = '198c7d8c-e010-45ce-a018-ec2d9a33f58f'

# Authenticate and get Workspace object
ia = InteractiveLoginAuthentication(tenant_id=tenant_id)
ws_name = 'automlbook'
subscription_id = '4d278f3d-b4fd-4fa2-86b6-d34b96bc888f'
resource_group = 'Foxy_Resources'
ws = Workspace.get(name=ws_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,
                   auth=ia)
print("After authenticating to the workspace with Interactive Authentication:", 
        "ws.name: " + ws.name, 
        "ws.resource_group: " + ws.resource_group, 
        "ws.location: " + ws.location, 
        "ws.subscription_id: " + ws.subscription_id, 
sep='\n')

In [ ]:
# (Commmented example code to Auth with Service Principal)
# (Purely to Validate this will work inside script...) 

# TODO? maybe? I could build a custom image with BuildKit, with "Secret mount type", and then have Azure get it out of Docker Repository
# Store authentication strings
# Store username and password to Service Principal in order to authenticate within the python script
# CRITICAL: You must have a file like this at resources/custom_env_vars_for_script_inside_docker_container (from project root)...
#       # Set AzureML Service Principle ID and Password
#       AML_PRINCIPAL_ID="<Principal ID, AKA clientId>"
#       AML_PRINCIPAL_PASS="<Principal Password, AKA clientSecret>"

# Authenticate with the Service Principal in order to get the Workspace object
# from azureml.core.authentication import ServicePrincipalAuthentication
# sp = ServicePrincipalAuthentication(tenant_id=tenant_id,
#                                     service_principal_id=kv.get_secret(name="localDockerAmlPrincipalId"), # clientId of service principal
#                                     service_principal_password=kv.get_secret(name="localDockerAmlPrincipalPass")) # clientSecret of service principal
# ws = Workspace.get(name=ws_name,
#                    subscription_id=subscription_id,
#                    resource_group=resource_group,
#                    auth=sp)
# print("After re-authenticating to the workspace with Service Principal", 
#         "ws.name: " + ws.name, 
#         "ws.resource_group: " + ws.resource_group, 
#         "ws.location: " + ws.location, 
#         "ws.subscription_id: " + ws.subscription_id, 
# sep='\n')

In [ ]:
# (Print statements to list available environments)
# envs = Environment.list(workspace=ws)

# for env in envs:
    # if env.startswith("AzureML"):
        # print("Name",env)
        # if None != envs[env].python.conda_dependencies:
            # print("packages", envs[env].python.conda_dependencies.serialize_to_string())


In [ ]:
# Create datastore, try getting datastore via Workspace object
datastore = Datastore.get_default(ws)
datastore_name = 'workspaceblobstore'
datastore = Datastore.get(ws, datastore_name)

In [ ]:
# (Only Uncomment and Run Once to register the Tabular OJSales Data) 
# Download Data Files to convert to a Tabular Dataframe we can more easily use

# import os
# from pathlib import Path
# 
# Open Dataset containing our Orange Juice sales data (requires pip install azureml-opendatasets)
# from azureml.opendatasets import OjSalesSimulated

# Create a dataset from the datastore of the Workspace
# oj_sales_files = OjSalesSimulated.get_file_dataset() 
# oj_sales = oj_sales_files.take(10) 

# IMPORTANT TIP! 
#  There are over 4,000 files in the OJ Sales Simulated Data Azure Open Dataset. 
#  Pulling all of them can result in extended training times.

# Create folder to hold OJ_Sales files
#
# In order to use file datasets, you first need to download them to your local Jupyter environment. 
# Then, you can read them in as a pandas dataframe by concatenating the files. 
# Download the 10 files to your newly created OJ_Sales folder with the following code: 

# folder_name = "OJ_Sales" 
# os.makedirs(folder_name, exist_ok=True)
# oj_sales.download(folder_name, overwrite=True)

# OJ_file_path = Path(folder_name).rglob('*.csv') 
# OJ_files = [x for x in OJ_file_path] 
# df = pd.concat((pd.read_csv(f) for f in OJ_files))

# print(df.head(20))

# Dataset.Tabular.register_pandas_dataframe(df, datastore, "automlbook OJSales Forecassting Training A")

In [ ]:
# Create a dataset from the datastore of the Workspace
dataset_name = 'automlbook Iris Multiclass Training A'
# dataset = Dataset.get_by_name(ws, dataset_name)
dataset = Dataset.get_by_name(ws, dataset_name, version = 'latest')
dataset_columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
target_column_name = 'species'

# Show a sample of the data in the dataset
print(dataset.take(10).to_pandas_dataframe())

# Turn Dataset into Pandas Dataframe, it is to be preprocessed
df = dataset.to_pandas_dataframe()

In [ ]:
# Verify number of classes of target column
num_classes = df[target_column_name].nunique()

expected_classes = 3
if (num_classes != expected_classes):
    raise Exception(target_column_name + " target field expected to have this many classes: ", expected_classes)


In [ ]:
# Add a weight_column to have AutoML give different weight to missed predictions of certain rows of data.
#     - A common use is to weigh missed classifications differently (based on class)
weight_column_name = 'weight_column'
df[weight_column_name] = np.where(True, 1, 0)
df[weight_column_name] = np.where(df[target_column_name] == 'Iris-setosa', 1, df[weight_column_name])
df[weight_column_name] = np.where(df[target_column_name] == 'Iris-versicolor', 100, df[weight_column_name])
df[weight_column_name] = np.where(df[target_column_name] == 'Iris-virginica', 100, df[weight_column_name])
#
# EXAMPLE: Assigning weight (to incorrectly predicting) of 1 to survivors and 0.1 to victims
#     - (so a missed prediction of a person surviving has weight 1)
#     - (and so a missed prediction of a person not surviving has weight 0.1)
#

In [ ]:
# (COMMENTED OUT) Bin an age column into four different age groups. (Under 15, 15-35, 35-60, and over 60)
# df['AgeUnder15'] = np.where(df.Age < 15, 1, 0) 
# df['Age15to34'] = np.where((df.Age > 14) & (df.Age < 35), 1, 0) 
# df['Age35to60'] = np.where((df.Age > 34) & (df.Age < 61), 1, 0) 
# df['AgeOver60'] = np.where(df.Age > 60, 1, 0)

# df = df.drop(['Age'],axis=1)

In [ ]:
# Partition Dataframe to get one for Numeric columns and one for Categorical columns
df_column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

##### BEGIN Create dataframe with numeric columns so that it contains all numbers that are preprocessed...
df_numeric_column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

# Copy df, keeping only float columns
df_float_column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
df_float_columns = []
if (len(df_float_column_names) != 0):
    df_float_columns = pd.DataFrame(df[df_float_column_names], dtype=np.float, columns=df_float_column_names)

# Copy df, keeping only int columns
df_int_column_names = []
df_int_columns = []
if (len(df_int_column_names) != 0):
    df_int_columns = pd.DataFrame(df[df_int_column_names], columns=df_int_column_names)

# Concatenate the numeric DataFrames
if (len(df_int_columns) > 0 and len(df_float_columns) > 0):
    df_numeric_columns = pd.concat([df_int_columns, df_float_columns], axis=1)
elif (len(df_int_columns) > 0):
    df_numeric_columns = df_int_columns
elif (len(df_float_columns) > 0):
    df_numeric_columns = df_float_columns
    
print('concatenated df_numeric_columns: ', df_numeric_columns)

##### BEGIN Create dataframe with categorical columns so that it contains all categorical data that is preprocessed...
df_categorical_column_names = []
print("df_categorical_column_names.count is " + str(len(df_categorical_column_names)))
df_categorical_columns = []
# Copy df, keeping only categorical columns
if (len(df_categorical_column_names) != 0):
    df_categorical_columns = pd.DataFrame(df[df_categorical_column_names], dtype=np.str, columns=df_categorical_column_names)
    print('df_categorical_columns: ', df_categorical_columns)

feature_column_names = [*df_numeric_column_names, *df_categorical_column_names]
print("feature_column_names: ", feature_column_names)

if (len(feature_column_names) == 0 or (len(df_categorical_columns) == 0 and len(df_numeric_columns) == 0)):
    raise Exception("There either are no feature columns names, or no feature columns found by those names.")

In [ ]:
# Combine the numeric DF with the categorical DF
# print("df[target_column_name] is ", df[target_column_name])
# print("df_numeric_columns is ", df_numeric_columns)
# print("df_numeric_columns.columns is ", df_numeric_columns.columns)
# print("df_categorical_columns is ", df_categorical_columns)
# print("df_categorical_columns.columns is ", df_categorical_columns.columns)

# Concatenate dfs to get DataFrame of all columns to submit to the training script
if (len(df_numeric_columns) > 0 and len(df_categorical_columns) > 0):
    dfs = [df[target_column_name], df[weight_column_name], df_numeric_columns, df_categorical_columns]
elif (len(df_numeric_columns) > 0):
    dfs = [df[target_column_name], df[weight_column_name], df_numeric_columns]
elif (len(df_categorical_columns) > 0):
    dfs = [df[target_column_name], df[weight_column_name], df_categorical_columns]
else:
    raise Exception("There are no feature columns available in the lists of them provided.")
# print("dfs is" + str(dfs))
# print('Before concatenation to dfTyped, df[\'target_column_name\']: ', df[target_column_name])
# print('Before concatenation to dfTyped, df_numeric_columns: ', df_numeric_columns)
dfTyped = pd.concat(dfs, axis=1)
print('dfTyped: ', dfTyped)

In [ ]:
# Split pre-transformation Data Frame into feature/target columns
# df_weight_column = pd.DataFrame(dfTyped[weight_column_name], columns=[weight_column_name])
df_weight_column = dfTyped[weight_column_name].to_frame()
df_x = dfTyped.drop([target_column_name, weight_column_name], axis=1)
df_y = dfTyped[target_column_name].to_frame()
# print("See df_x", df_x)
print("See df_y", df_y)

In [ ]:
# Create preprocessor to preprocess numeric and categorical columns (with Transfomer API via ColumnTransformer)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', one_hot_encoder)])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, df_numeric_column_names),
        ('cat', categorical_transformer, df_categorical_column_names)])

In [ ]:
# (Validate the Pipeline will function once passed to the script)

# Get the preprocessed Data Frame columns in a list
if (len(df_categorical_columns) != 0): # There are categorical columns
    one_hot_encoder.fit(df_categorical_columns)
    # Get new One Hot Encoded column names
    encoded_categorical_column_names = one_hot_encoder.get_feature_names(df_categorical_column_names)
    encoded_feature_names = [*df_numeric_column_names, *encoded_categorical_column_names]
else: # There are no categorical columns
    encoded_categorical_column_names = []
    encoded_feature_names = [*df_numeric_column_names]


print(str(encoded_feature_names))

preprocessor.fit(df_x)

In [ ]:
# Apply the preprocessor to the feature columns (then concat the weight_column Dataframe to it)
df_x = preprocessor.transform(df_x)
df_x = pd.DataFrame(df_x, columns=encoded_feature_names)
df_x = pd.concat([df_x, df_weight_column], axis=1)
print('df_x right before registration, with weight_column concatenated.\n', df_x)

In [ ]:
# Register Pandas Dataframe of base df_x and df_y
Dataset.Tabular.register_pandas_dataframe(df_x, datastore, "Iris MultiClass Feature Column Data for train_test_split usage (Docker Environment)")
Dataset.Tabular.register_pandas_dataframe(df_y, datastore, "Iris MultiClass Target Column Data for train_test_split usage (Docker Environment)")

In [ ]:
# Split data into training and test data, register the resulting Datasets with Azure
from sklearn.model_selection import train_test_split

# Split the data
# What you need to pass to train_test_split...
# ... I need X and Y dataframe, X just with target missing, Y just with target column present
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,
                                                    test_size=0.2,
                                                    random_state=0)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
print(test_data)

# print("See y_test", y_test)
# print("See y_test.columns.tolist()", str(y_test.columns.tolist()))
# print("See y_test.values.tolist() to pass as true_ys to ExplanationClient.upload_model_explanation()", y_test.values.tolist())
# print("See y_test.values.ravel() to pass as true_ys to ExplanationClient.upload_model_explanation()", y_test.values.ravel())
# print("See y_test.values.tolist().flatten() to pass as true_ys to ExplanationClient.upload_model_explanation()", y_test.values.flatten())

# Register the split (by whether target column) training and test datasets
X_train_registered_name = "Iris MultiClass Feature Column Data for training (Docker Environment)"
X_test_registered_name = "Iris MultiClass Feature Column Data for testing (Docker Environment)"
y_train_registered_name = "Iris MultiClass Target Column Data for training (Docker Environment)"
y_test_registered_name = "Iris MultiClass Target Column Data for testing (Docker Environment)"
# Dataset.Tabular.register_pandas_dataframe(X_train, datastore, X_train_registered_name)
Dataset.Tabular.register_pandas_dataframe(X_test, datastore, X_test_registered_name)
# Dataset.Tabular.register_pandas_dataframe(y_train, datastore, y_train_registered_name)
Dataset.Tabular.register_pandas_dataframe(y_test, datastore, y_test_registered_name)

# Register the combined (feature and target columns) training and test datasets
train_data_registered_name = "Iris MultiClass Training Data (Docker Environment)"
test_data_registered_name = "Iris MultiClass Training Test Data (Docker Environment)"
Dataset.Tabular.register_pandas_dataframe(train_data, datastore, train_data_registered_name)
Dataset.Tabular.register_pandas_dataframe(test_data, datastore, test_data_registered_name)

trainTestDataSetNames = [X_train_registered_name, X_test_registered_name, y_train_registered_name, y_test_registered_name, train_data_registered_name, test_data_registered_name]

In [ ]:
# Save feature names and create TabularExplainer with them
features=[*df_numeric_column_names, *encoded_categorical_column_names]

In [ ]:
# Encode Experiment script arguments list into a string like '["a","b"]'

# Encode numeric column names list
temp_column_names = df_numeric_column_names.copy()
for x in range(len(temp_column_names)):
        temp_column_names[x] = '"{}"'.format(temp_column_names[x])
numericFeatureNamesEncoded = "[{}]".format(",".join(temp_column_names))
# print("numericFeatureNamesEncoded:", numericFeatureNamesEncoded)

# Encode categoric column names list
temp_column_names = encoded_categorical_column_names.copy()
for x in range(len(temp_column_names)):
        temp_column_names[x] = '"{}"'.format(temp_column_names[x])
categoricFeatureNamesEncoded = "[{}]".format(",".join(temp_column_names))
# print("categoricFeatureNamesEncoded:", categoricFeatureNamesEncoded)

# Encode split dataset names list
for x in range(len(trainTestDataSetNames)):
        trainTestDataSetNames[x] = '"{}"'.format(trainTestDataSetNames[x])
trainTestDataSetNamesEncoded = "[{}]".format(",".join(trainTestDataSetNames))
print("splitDatasetNamesEncoded:", trainTestDataSetNamesEncoded)

In [ ]:
# Set Dockerfile, Set Local Docker Environment up (with System Managed Dependencies, via Conda)
#
# Learn about Environment and how to use a Docker Environment here:
#       https://docs.microsoft.com/en-us/azure/machine-learning/concept-environments
#       https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment(class)?view=azure-ml-py
#       ! IMPORTANT: https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/
#
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

docker_env = Environment("docker-env")
# Editing a run configuration property on-fly.
docker_env.python.user_managed_dependencies = False
# Use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param.
#           https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.runconfig.dockerconfiguration?view=azure-ml-py
docker_config = DockerConfiguration(use_docker=True)
print("initial base image from base docker-env Environment: ", docker_env.docker.base_image)

# Specify docker steps as a string. 
dockerfile = r"""
FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220314.v1

ARG DEBIAN_FRONTEND=noninteractive
ARG ACCEPT_EULA=Y
RUN ls -altr
RUN ls -altr .dockerenv
RUN ls -altr dev
RUN ls -altr home
RUN ls -altr run
RUN apt-get update -y && apt-get upgrade -y &&\
    apt-get install -y build-essential \
                       cmake \
                       curl \
                       gfortran \
                       git \
                       jupyter \
                       libatlas-base-dev \
                       libblas-dev \
                       libbz2-dev \
                       libffi-dev \
                       libgdbm-dev \
                       liblapack-dev \
                       liblzma-dev \
                       libncurses5-dev \
                       libncursesw5-dev \
                       libreadline-dev \
                       libsqlite3-dev \
                       libssl-dev \
                       libxml2-dev \
                       libxmlsec1-dev \
                       llvm \
                       lzma \
                       lzma-dev \
                       make \
                       tcl-dev \
                       tk-dev \
                       wget \
                       xz-utils \
                       zlib1g-dev

RUN conda -V
RUN echo "Hello from custom container!" > ~/hello.txt
# RUN export PIP_LOG="/tmp/pip_log.txt" && touch ${PIP_LOG} && tail -f ${PIP_LOG} & conda env create -f "conda.yml" && killall tail && rm ${PIP_LOG}
"""

# NOTE: you can pass Dockerfile string to docker build command via stdin like this:
#
# sudo docker build -t myimage:latest -<<EOF
# FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220314.v1
# RUN echo "hello world"
# EOF


# Set base image to None, because the image is defined by dockerfile.
docker_env.docker.base_image = None
# Use the Dockerfile string and build image based on it with this code from above (move it down here)
docker_env.docker.base_dockerfile = dockerfile

#       For help, try reading this
#                    - https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/#custom-docker-image--dockerfile
#       May also try using other base Docker images from these container registries
#                - https://github.com/microsoft/containerregistry
#                - https://github.com/Azure/AzureML-Containers
# env.docker.base_image = '<image-name>'
# env.docker.base_image_registry.address = '<container-registry-address>'
# env.docker.base_image_registry.username = '<acr-username>'
# env.docker.base_image_registry.password = os.environ.get("CONTAINER_PASSWORD")
#
# TODO? use this link to get username and password from Azure KeyVault:
#           https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/#use-keyvault-to-pass-secrets
# TODO? use this code to set username and password:
# # Retrieve username and password from the workspace key vault
#       env.docker.base_image_registry.username = ws.get_default_keyvault().get_secret("username")  
#       env.docker.base_image_registry.password = ws.get_default_keyvault().get_secret("password")

In [ ]:
# Specify conda and pip dependencies for (Docker) environment
# DONE get 'azureml-train-automl' installed, the docker build times out for some reason when I added that to pip_packages
#           Help: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-environments

conda_packages = ['pip'
                 ]

pip_packages =   ['azure-core==1.24.1',
                  'azure-storage-queue==12.3.0',
                  'cffi==1.15.0',
                  'cryptography==37.0.3',
                  'dataclasses==0.6',
                  'google-auth==2.8.0',
                  'opencensus==0.9.0',
                  'opencensus-ext-azure==1.1.4',
                  'portalocker==2.4.0',
                  'toolz==0.11.2',
                  'azureml.train.automl==1.43.0',
                  'azureml.interpret==1.43.0',
                  'python-dotenv==0.20.0',
                  'rsa==4.8',
                 ]

condaDependencies = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)
docker_env.python.conda_dependencies = condaDependencies

In [ ]:
# Register the Docker Environment and build the Docker image locally
registered_docker_env = docker_env.register(ws)
print(registered_docker_env)
# Need to enable non-root docker user usage of docker for this local build of the image to work, see guide:
#           (Actually, this was not enough it seems, I am getting no feedback here, hm.)
#           (Then I tried: sudo chmod 777 /var/run/docker.sock, I think 770 is enough because of the docker group owning /var/run/docker.sock at 660 initially)
      # (https://docs.docker.com/engine/install/linux-postinstall/#manage-docker-as-a-non-root-user)

registered_docker_env.save_to_directory('environment_out', overwrite=True)
# If this fails, make sure docker service is running
registered_docker_env.build_local(ws, useDocker=True, pushImageToWorkspaceAcr=True)

In [ ]:
# Prepare to run AutoML Training Experiment in Docker Environment (With Docker running on local device)
# TODO? maybe turn off this featurization to leave previous featurization or to enhance featurization yourself

from azureml.core import ScriptRunConfig
import datetime

# Set ScriptRunConfig arguments
compute_target = 'local'
source_directory = './scripts'
script_name = 'ojSalesDockerForecastingTrainingAutoML.py'
suffix = 'local-' + str(datetime.datetime.now())
suffix = suffix.replace(' ', '_') # Clean up datetimestamp
suffix = suffix.replace(':', '-') 
out_model_file_name = 'AutoMLMultiClassification_Iris_BestModel_Docker_{}.pkl'.format(suffix)
# set output file name like 'AutoMLMultiClassification_Iris_BestModel_Docker_2022-04-17 21:40:36.114550.pkl'
expiriment_name_prefix = "Iris_Docker_MultiClassification_Training"
automlconfig_experiment_name = expiriment_name_prefix + '_AutoML'
scriptrunconfig_experiment_name = expiriment_name_prefix + '_ScriptRunConfig'

script_arguments = [
"--tenant-id", tenant_id,
"--ws-name", ws_name,
"--subscription-id", subscription_id,
"--resource-group", resource_group,
"--datastore-name", datastore_name,
"--out-model-file-name", out_model_file_name,
"--target-column-name", target_column_name,
"--numeric-feature-names", numericFeatureNamesEncoded,
# Pass list encoded as a comma-separated string, containing the raw feature names
# like '["a","b"]'
"--categoric-feature-names", categoricFeatureNamesEncoded,
# Pass list encoded as a comma-separated string, containing the name of each dataset (X_train, X_test, y_train, y_test)
# like '["a","b"]'
"--x-train-test-y-train-test-combined-train-test", trainTestDataSetNamesEncoded,
"--num_classes", num_classes,
"--weight_column_name", weight_column_name,
"--automlconfig_experiment_name", automlconfig_experiment_name
]
print("ScriptRunConfig arguments: ", script_arguments)
scriptRunConfig = ScriptRunConfig(
        source_directory=source_directory,
        script=script_name,
        arguments=script_arguments,
        environment=registered_docker_env,
        docker_runtime_config=docker_config)

In [ ]:
# Submit Experiment Run to Docker environment
#
# (see more on use of Docker environment: 
#   https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/train-on-local/train-on-local.ipynb
# )
import subprocess
# TODO? Add minconda bin to path within docker container? "/home/johna/miniconda3/bin:"

# import getpass
# If you need, you can get a password from user input (Notebook pauses to show a prompt here)
# password = getpass.getpass()

# Check if Docker is installed and Linux containers are enabled
if subprocess.run("docker -v", shell=True).returncode == 0:
    subprocess.run("service docker status", shell=True)


    # This StackOverflow page will help you run the docker commands with sudo if that is necessary for you: 
    # https://askubuntu.com/questions/155791/how-do-i-sudo-a-command-in-a-script-without-being-asked-for-a-password
    #
    # (NOTE: These snippets from the link will allow you to have the docker commands run with sudo)
    #
    # Create script with the command to run with sudo, for example:
    # docker system info
    #
    # Run sudo chown and chmod commands to grant access of the file to root 
    # sudo chown root:root ~/docker_system_info.sh
    # sudo chmod 700 ~/docker_system_info.sh
    #
    # Run sudo visudo and insert a line below the line `%sudo   ALL=(ALL:ALL) ALL`
    # [username]  ALL=(ALL) NOPASSWD: /home/[username]/docker_system_info.sh
    #
    # Then call the python script with your python subprocess command
    # p = subprocess.run('sudo ~/docker_system_info.sh', shell=True)

    # out_docker_system_info = subprocess.run("docker system info", shell=True)
    out_docker_system_info = subprocess.check_output('~/docker_system_info.sh', shell=True).decode('ascii')
    # out_docker_system_info = subprocess.check_output('sudo su && ~/docker_system_info.sh', shell=True).decode('ascii')
    print(out_docker_system_info)
        #           [Install Ubuntu](https://docs.docker.com/engine/install/ubuntu/)
        #           [Uninstall Docker Engine](https://docs.docker.com/engine/install/ubuntu/#uninstall-docker-engine)
        #           WARNING! When I ran this command there is a failure to uninstall and purge all docker engine apt-get packages `sudo apt-get purge docker-ce docker-ce-cli containerd.io docker-compose-plugin`
        #                   AND TO RESOLVE (EXTRA CAPS) !!!THIS!!! THING, I DID THE FOLLOWING...
        #
        # Get /usr/bin/docker: Permission denied? (was not enough for me)
        #       See (https://adamtheautomator.com/docker-permission-denied/)
        #           (perhaps I need apt dependencies added to Dockerfile string I passed(??))
        #       Along with  (https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/)
        # TODO? update for my laptop: still not getting any logs past "nvidia-docker is installed on the target. Using nvidia-docker for docker operations"
        # 
        # NOTE - NEXT... Try using custom image from your own Docker image repository
        #                   (perhaps allow docker to run by johna user normally, without any black magic)
        # 
        # LAST DITCH EFFORTS - If you are stuck, uninstall and install docker, then install different Linux version for WSL, then try reinstall WSL
        #
        # NOTE - How to get past error: (perhaps uninstall nvidia-docker)
        #           nvidia-docker is installed on the target. Using nvidia-docker for docker operations.
        # 
        # You may want to follow this guide to install the Docker engine into Ubuntu:
        #           (https://docs.docker.com/engine/install/ubuntu/)
        #   Post install steps:
        #           (https://docs.docker.com/engine/install/linux-postinstall/#manage-docker-as-a-non-root-user)
        #   Alternative way to install:
        #            https://github.com/docker/docker-install
        #
        # Run Experiment in Docker environment
        # (NOTE: If you get any errors, in AMLS go to Jobs -> Click Experiment then Run from list -> Look for "Environment", 
        #           There should be a hyperlink to a page for the Environment used for that run!
        #           There should be a Docker Build Log you can access
        #           You should be able to trigger a build of the Docker image from the Environment's main page
        # )
    experiment = Experiment(workspace=ws, name=scriptrunconfig_experiment_name)
    # NOTE: I previously got an error message including "GPU", because of a --gpu flag used the instructions at this link to get past that:
    #           (https://docs.nvidia.com/cuda/wsl-user-guide/index.html)
    # NOTE: If script is failing at authentication, follow this link for help:
    #           (https://docs.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication)

    # ScriptRunConfig usage to create a Run
    ScriptRunConfig_run = experiment.submit(scriptRunConfig)
    RunDetails(ScriptRunConfig_run).show()

In [ ]:
# Get the engineered feature names
# (While Experiment script runs, Validate the Engineered Feature Explanation will function inside the script)

# Split training features into numeric and categoric dataframes
# numeric_X_test = pd.DataFrame(X_test[df_numeric_column_names], dtype=np.str, columns=df_numeric_column_names)
# categoric_X_test = pd.DataFrame(X_test[df_categorical_column_names], dtype=np.str, columns=df_categorical_column_names)
# # Fit and Run the numeric and categoric ColumnTransformers on the split dataframes to perform feature engineering
# preprocessor.transformers[0][1].fit(numeric_X_test)
# preprocessor.transformers[1][1].steps[0][1].fit(categoric_X_test)
# numeric_X_test_preprocessed = preprocessor.transformers[0][1].transform(numeric_X_test)
# numeric_X_test_preprocessed = pd.DataFrame(numeric_X_test_preprocessed, dtype=np.float, columns=df_numeric_column_names)
# categoric_X_test_preprocessed = preprocessor.transformers[1][1].steps[0][1].transform(categoric_X_test)
# # Fit OneHotEncoder
# preprocessor.transformers[1][1].steps[1][1].fit(categoric_X_test_preprocessed)
# # Get new One Hot Encoded column names
# print(categoric_X_test_preprocessed)
# print(df_categorical_column_names)
if (0 != len(df_categorical_column_names)):
    df_encoded_categorical_column_names = preprocessor.transformers[1][1].steps[1][1].get_feature_names(df_categorical_column_names)
else:
    df_encoded_categorical_column_names = []
# print("df_encoded_categorical_column_names", df_encoded_categorical_column_names)
# # Transform categoric, null-imputed features with fitted OneHotEncoder
# categoric_X_test_preprocessed = preprocessor.transformers[1][1].steps[1][1].transform(categoric_X_test_preprocessed)
# # Turn preprocessed categoric features into a DataFrame
# categoric_X_test_preprocessed = pd.DataFrame(categoric_X_test_preprocessed, dtype=np.float64, columns=df_encoded_categorical_column_names)

# # Combine the numeric DF with the categorical DF to submit to the AutoML training experiment
# X_test_preprocessed_list = [numeric_X_test_preprocessed, categoric_X_test_preprocessed]
# X_test_preprocessed = pd.concat(X_test_preprocessed_list, axis=1)


# Save engineered features' names to create TabularExplainer with them
engineeredFeatures=[*df_numeric_column_names, *df_encoded_categorical_column_names]
print(engineeredFeatures)

In [ ]:
# # TODO Somehow wait until script run completion ... (Manually terminate this cell because exception keeps ocurring) Wait for completion of script
# from azureml.core import Model
# import sklearn
# import time
# while True:
#   try:
#     ScriptRunConfig_run.wait_for_completion()
#     break
#   except:
#       print ("encountered exception waiting for completion of ScriptRunConfig_run, waiting and trying again...") 
#       time.sleep(60)

In [ ]:
# TODO? Get the model metrics somehow and plot output Best Model metrics

# Get all metris logged in the run
metrics = ScriptRunConfig_run.get_metrics()
print("metrics: ", metrics)

# Get the metrics that were logged from the run of the training script
# print("metrics['accuracy']: " + str(metrics['accuracy']))
# print("metrics['f1']: " + str(metrics['f1']))

# %matplotlib inline
# import matplotlib
# import matplotlib.pyplot as plt
# plt.plot(metrics['accuracy'], metrics['f1'], marker='o')
# plt.ylabel("f1")
# plt.xlabel("accuracy")
#
# You can also list all the files that are associated with this run record
#
print("Here are the files associated with the Azure AutoML Run: ", ScriptRunConfig_run.get_file_names())

In [ ]:
# For the Experiment Run that was ran, Get Global Explanations (Downloaded with the Experiment Run object!) # TODO? Download model explanations
# IMPORTANT: this step will fail until you copy the explanation_id values from the printed list of explanations
# from azureml.interpret import ExplanationClient

# client = ExplanationClient.from_run(ScriptRunConfig_run)

# Get model explanation data
# TODO? Get the explanations, for now this client.list_model_explanations() is returning empty
# print(client.list_model_explanations())
# IMPORTANT: this step will fail until you copy the explanation_id values from the printed list of explanations
# engineered_global_explanation_test = client.download_model_explanation(explanation_id='7e81718e-4481-4dcb-a269-0db977df3436')
# raw_explanation = client.download_model_explanation(explanation_id='86861485-00ae-42f0-8bab-3ae41556c6a9')


# Or only get the top k (e.g., 4) most important features with their importance values
# explanation = client.download_model_explanation(top_k=4)

# global_importance_values = engineered_global_explanation_test.get_ranked_global_values()
# global_importance_names = engineered_global_explanation_test.get_ranked_global_names()
# print('global importance values: {}'.format(global_importance_values))
# print('global importance names: {}'.format(global_importance_names))

In [ ]:
# TODO? Stop the failures here and download the best model from the AutoML run that's inside the script (then visualize the explanation)
# Download the Model from AzureML and Visualize Explanations with it 
from raiwidgets import ExplanationDashboard
from azureml.core.model import Model
import joblib
import jinja2

# print('Model.list(ws)', Model.list(ws))

# Download the Model from Azure

# Use Model.download and joblib.load()
# remote_model_obj = Model(ws, out_model_file_name)
# print('Name:', remote_model_obj.name)
# print('Version:', remote_model_obj.version)
# remote_model_path = remote_model_obj.download(exist_ok = True)
# downloaded_model = joblib.load(remote_model_path)

# BEGIN Access "Local Explanations", uncomment these lines if you want to do that here...
# (Local Explanation meaning "of individual predictions") 
# from interpret.ext.blackbox import TabularExplainer
# "features" and "classes" fields are optional
# explainer = TabularExplainer(downloaded_model,
                            #  X_test,
                            #  features=features)

# Get explanation for the first few data points in the test set
# local_explanation = explainer.explain_local(X_test[0:5])
# Sorted feature importance values and feature names
# sorted_local_importance_names = local_explanation.get_ranked_local_names()
# print('sorted_local_importance_names: ', sorted_local_importance_names)
# print('len(sorted_local_importance_names): ', len(sorted_local_importance_names))
# sorted_local_importance_values = local_explanation.get_ranked_local_values()
# print('sorted_local_importance_values: ', sorted_local_importance_values)
# print('len(sorted_local_importance_values): ', len(sorted_local_importance_values))
# COOL THING TO DO: Sometime could get local explanation of specific data points uploaded, downloaded, and visualized as well...
# END Access "Local Explanations"

# Visualize explanations
# Be sure to pass dataset=(test feature columns Dataframe) and true_y=(test predicted column Dataframe)
#       1) getting the raiwidgets thing working
#       2) see README at https://github.com/interpretml/interpret
# ExplanationDashboard(engineered_global_explanation_test, downloaded_model, dataset=X_test, true_y=y_test)